In [27]:
import os
import cv2
import numpy as np
from imutils import paths
from PIL import Image

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
datapath = r"C:\Users\venug\Downloads\55\data"
outputmodel = r"C:\Users\venug\Downloads\55\vc\VideoClassificationModel"
outputlabelbinarizer = r"C:\Users\venug\Downloads\55\vc\videoclassificationbinarizer"
epoch=25

In [3]:
Sports_Labels= set(['badminton','chess','football'])
print("image being loaded")
pathToImages = list(paths.list_images(datapath))
data=[]
labels=[]
for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in Sports_Labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image,(224,224))
    data.append(image)
    labels.append(label)
data = np.array(data)
labels = np.array(labels)

image being loaded


In [7]:
from sklearn.preprocessing import LabelBinarizer
data = np.array(data)
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
traininAugmentation = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,  # Corrected typo: Changed 'O' to '0'
    horizontal_flip=True,
    fill_mode='nearest'  # Corrected typo: Replaced '-' with '='
)
validationAugmentation = ImageDataGenerator()

mean = np.array([123.68, 116.779, 103.939],dtype="float32") # Corrected syntax: Square brackets for array elements
traininAugmentation.mean = mean
validationAugmentation.mean = mean  # Corrected typo: Replaced 'traininAugmentation' with 'trainingAugmentation'

In [11]:
from keras.optimizers import SGD

In [12]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay

epoch = 25
learning_rate = 0.0001
momentum = 0.9

# Define the learning rate schedule
lr_schedule = ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=10000, # This should be adjusted based on your training data size
    decay_rate=0.9 # Adjust the decay rate as needed
)

# Create the optimizer with the learning rate schedule
opt = SGD(learning_rate=0.0001, momentum=0.9)

In [ ]:
from sklearn.model_selection import train_test_split
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [15]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, Dropout, AveragePooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define your data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # Use preprocess_input for ResNet50
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input # Use preprocess_input for ResNet50
)

# Use the datapath variable for flow_from_directory
train_generator = train_datagen.flow_from_directory(
    datapath,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)

# Load the ResNet50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = AveragePooling2D(pool_size=(7, 7))(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(24, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

epochs = 25
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs
)

Found 2188 images belonging to 3 classes.


C:\Users\venug\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


69/69 ━━━━━━━━━━━━━━━━━━━━ 294s 4s/step - accuracy: 0.5849 - loss: 1.5960


In [30]:
# version 1:(checked)
import pickle
model.save('VideoClassificationModel.keras')

# Assuming 'lb' is your label binarizer
# Save the label binarizer
lbinarizer = open(r"C:\Users\venug\Downloads\55\vc\videoclassificationbinarizer.pickle", "wb")
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()